# GUI Development File

## Need to make a readme file

### Current State

Current Goal: Remove hardcoding in GUI definition

Goals:

- [x] Initial creation of the GUI and widgets
  - [x] URL entry
  - [x] Download button
  - [x] Verification button
  - [x] Directory entry
  - [x] Directory browse Button
  - [x] File name entry
  - [x] Quality dropdown
- [x] Verification button Functionality
  - [x] Get video info
    - [x] Warning upon no entry
    - [x] Error catching for non-video link
    - [x] Error catching for extraction issues
    - [x] Generic error catching
  - [x] Display video info to user
    - [x] create function to make readable formats for the dropdown
    - [x] sort the formats from best to worst quality
  - [x] Update the file name entry with the video title
  - [x] Display error to user upon failure
- [ ] Directory Selection Functionality
  - [ ] Enable browse button to open folder selection window
  - [ ] Set default path in entry to user/video directory
    - [ ] Ensure validity for Windows, Mac OS, and Linux users
- [ ] Download button
  - [ ] Final verification that inputs have not changed
    - [ ] URL
    - [ ] Title
    - [ ] Directory
    - [ ] Format/Quality
  - [ ] Format yt_dlp input
    - [ ] format = user selected format
    - [ ] outtmpl = user selected directory + file name
  - [ ] Begin downloading from the URL
  - [ ] Progress tracking
    - [ ] Progress bar
- [ ] Create standalone .exe file
  - [ ] Ensure no need to external dependencies
  - [ ] way forward tbd

In [4]:
# GUI that will allow for video downloads via webscraping using the yt_dlp library

######################################################################
############################### Imports ##############################
######################################################################
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinter import ttk
import yt_dlp
from pathlib import Path


######################################################################
######################### Global Variables ############################
######################################################################
current_video_info = None  # Store current video information
download_thread = None     # Store download thread reference

######################################################################
######################### Callback Functions #########################
######################################################################

def get_format_info(info):
    # Extract and create legible version of the vailable video formats
    processed_formats = [] # Empty array for final formats
    raw_formats = info.get('formats',[]) # retrieve the available formats from the input
    
    for fmt in raw_formats:
        # extract necessary information from the raw format information
        format_info = {
            'format_id': fmt.get('format_id','unknown'),
            'ext': fmt.get('ext', 'unknown'), # file extension
            'height': fmt.get('height'), # video height
            'width': fmt.get('width'), # video width
            'fps': fmt.get('fps'), # frames per secon
            'vcodec': fmt.get('vcodec','unknown'), # video codec
            'acodec': fmt.get('acodec','unknown'), # video codec
            'filesize': fmt.get('filesize'), # file size in bytes
            'quality': fmt.get('quality'), # quality rating (best, high, low, etc.)
            'has_video': fmt.get('vcodec') != 'none', # True if contains video
            'has_audio': fmt.get('acodec') != 'none' # True if contains audio
        }
        
        # Creates the description for the video
        if format_info['height']:
            # define quality (1080p, 720p, etc.) followed by extension (.mp4, .webm, etc.)
            description = f"{format_info['height']}p - {format_info['ext']}"
            if format_info['fps']:
                # add fps to quality description if available
                description += f" ({format_info['fps']}fps)"
        # Creates description if only audio is available for the selected format
        elif format_info['has_audio'] and not format_info['has_video']:
            description = f"Audio Only - {format_info['ext']}"
        else:
            description= f"{format_info['ext']}"
        
        # add the description to the format_info dictionary
        format_info['description'] = description
        
        # append the format_info dictionary into the array of processed_formats
        processed_formats.append(format_info)
        
    # sort formats from highest quality to lowest quality
    processed_formats.sort(key=lambda x: (
        x['has_video'] and x['has_audio'], # combined video and audio formats
        x['height'] or 0, # then sort by height quality
        x['quality']
    ), reverse=True)
    
    return processed_formats

def get_video_info(url):
    # Verify that input link contains a video and retrieve video information
    ydl_opts = { #yt_dlp options dictionary
        'quiet': True, # supress outputs
        'no_warnings': True # supress warnings
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            
            info = ydl.extract_info(url, download=False) # get info from link
            
            formats = get_format_info(info) # convert quality to readable format
            
            if info:                
                
                # Create video info dictionary if a video can be found on the link
                video_info = {
                    'title': info.get('title','Unknown'),
                    'duration': info.get('duration',0),
                    'uploader': info.get('uploader','Unknown'),
                    'upload_date': info.get('upload_date',''),
                    'formats_available': formats,
                    'format_count': len(formats),
                    'best_format': formats[0] if formats else None, # recommended format
                    'original_url': url
                }
                return True, video_info
            else:
                # else return no video information found as output
                return False, "No video information found"
            
    # Download exception for when: 
    # 1) video is geographically restricted
    # 2) The video requires additional authentication
    # 3) The video has been removed or made private
    # 4) Network connectivity issues
    except yt_dlp.DownloadError as e:
        return False, f"Download error: {str(e)}"
    
    # Extractor exception for when:
    # 1) website page structure is not as expected
    # 2) URL appears to be a video link but video cannot be found
    # 3) Unexpected data formats
    # 4) Website has anti-scraping measures
    except yt_dlp.ExtractorError as e:
        return False, f"Extractor error: {str(e)}"
    
    # catch all exception
    except Exception as e:
        return False, f"Unexpected error: {str(e)}"
    
def sanitise_filename(filename):
    # Remove or replace characters that are invalid in filenames
    # Replace invalid characters with underscores
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        filename = filename.replace(char,'_')
    
    # Remove leading/trailing dots and spaces
    filename = filename.strip('. ')
    
    # Limit length to avoid filesystem issues
    if len(filename) > 200:
        filename = filename[:200]
    
    return filename

def get_default_download_directory():
    # Get the default Videos directory 
    # should be universal across most OS, but will create a directory if one does not already exist
    try:
        videos_dir = Path.home() / 'Videos'
        
        # Create Directory if does not exist
        videos_dir.mkdir(exist_ok=True)
        return str(videos_dir)
    
    except:
        # Fallback to home directory
        return str(Path.home())
        

def verify():
    global current_video_info # store video info globally to prevent refetching of data during download
    
    # extract the supplied web address and ignore spaces at the start and end of the string
    url = web_address.get().strip() 
    
    # warning and escape if there is no input
    if not url:
        messagebox.showwarning("Warining", "Please enter a URL first.")
        return
    
    # Show loading message
    verifyButton.config(text="Getting Info...",state="disabled")
    window.update()
    
    success, result = get_video_info(url)
    
    # Reset button
    verifyButton.config(text="Verify", state="normal")

    if success:
        # Store video info into global variable
        current_video_info = result
        
        # show success message video info
        info_text = f"Video information retrieved!\n\n"
        info_text += f"Title: {result['title']}\n"
        info_text += f"Uploader: {result['uploader']}\n"
        if result['duration']:
            minutes = result['duration'] // 60
            seconds = result['duration'] % 60
            info_text += f"Duration: {minutes}:{seconds:02d}\n"
        info_text += f"Available formats: {result['format_count']}"        
        
        messagebox.showinfo("Video Info Retrieved", info_text)
        
        # Extract a list of the readable formats to display in the combobox
        combo_values = []
        for FormatOption in result.get("formats_available"):
            combo_values.append(FormatOption.get('description'))

        # Update the Format Combobox
        FormatCombo.config(values=combo_values,
                          state='readonly')
        if combo_values:
            FormatCombo.current(0) # Set current value to the best option
        
        # If no title is already set, then input the title from the video info
        if not FilenameEntry.get() and result['title']:
            FilenameValue.set(result['title'])
        
        # Set Download Button to Active
        DownloadButton.config(state='normal')

    else:
        # Wipes global video info to prevent any outdated information being used
        current_video_info = None
        
        # Show error message
        messagebox.showerror("Information Retrieval Failed", f"Could not get video info:\n\n{result}")

def browse_directory():
    # Open file dialog to select download directory
    selected_directory = filedialog.askdirectory(
        title="Select Download Directory",
        initialdir=DirectoryValue.get() or get_default_download_directory()
    )
    
    # Set the directory to the one selected by the user
    if selected_directory:
        DirectoryValue.set(selected_directory)
        
    return
    
def download():
    ######################## NOT CORRECT ATM ########################
    ydl_opts = {
    'format': 'quality slection',
    'outtmpl': r'directory location'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([URL])


######################################################################
####################### GUI Creation & Widgets #######################
######################################################################

# Window Creation
window = tk.Tk()
window.minsize(600, 250)
window.title("Video_Downloader.exe")

#Web address entry
linkLabel = tk.Label(text='Enter Link Here:',
                     font=('Arial',16)
                     )
linkLabel.pack(side='top',anchor='nw',padx=20,pady=5)

web_address = tk.Entry(window,
                       width=50,
                       font=('Arial',14),
                       justify='center'
                       )
web_address.pack(side='top',anchor='nw',padx=20,pady=5)

# Download and Verify Buttons
buttonFrame = tk.Frame(window)
buttonFrame.pack(side='top',anchor='n',padx=20)

verifyButton = tk.Button(buttonFrame,
                         text='Verify',
                         font=('Arial',16),
                         command=verify)
verifyButton.pack(side='left',padx=5)

DownloadButton = tk.Button(buttonFrame,
                           text='Download',
                           font=('Arial',16),
                           command=download,
                           state="disabled")
DownloadButton.pack(side='left',padx=5)

# Directory Select
DirectoryFrame = tk.Frame(window)
DirectoryFrame.pack(side='top', anchor='nw', padx=20, pady=5)
DirectoryLabel = tk.Label(DirectoryFrame, text="Download Directory:", font=('Arial', 12))
DirectoryLabel.pack(side='left')
DirectoryValue = tk.StringVar()
DirectoryEntry = tk.Entry(DirectoryFrame, 
                          textvariable=DirectoryValue,
                          width=39,
                          font=("Arial",12)
)
DirectoryEntry.pack(side='left',padx=5)
BrowseButton = tk.Button(DirectoryFrame, text="Browse", command=browse_directory)
BrowseButton.pack(side='left')

# File Name Definition
FilenameFrame = tk.Frame(window)
FilenameFrame.pack(side='top',anchor='nw',padx=20,pady=5)
FilenameLabel = tk.Label(FilenameFrame, text="File Name:", font=('Arial', 12))
FilenameLabel.pack(side='left')
FilenameValue = tk.StringVar()
FilenameEntry = tk.Entry(FilenameFrame, 
                          textvariable=FilenameValue,
                          width=52,
                          font=("Arial",12)
)
FilenameEntry.pack(side='left',padx=5)

# Format Select
FormatFrame = tk.Frame(window)
FormatFrame.pack(side='top', anchor='nw', padx=20, pady=5)
FormatLabel = tk.Label(FormatFrame,
                       text='Select Format/Quality Options: ',
                       font=("Arial",12))
FormatLabel.pack(side='left')

FormatCombo = ttk.Combobox( # Dropdown box for the format options
                           FormatFrame,
                           state='disabled',
                           font=('Arial',12),
                           width=35
)
FormatCombo.pack(side='left')


# main loop
window.mainloop()